In [44]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

In [45]:
df = pd.read_csv('./data/Titanic-Dataset.csv')
df.shape

(891, 12)

In [46]:
df.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
55,56,1,1,"Woolner, Mr. Hugh",male,NaN,0,0,19947,35.5000,C52,S
74,75,1,3,"Bing, Mr. Lee",male,32.0,0,0,1601,56.4958,NaN,S
77,78,0,3,"Moutal, Mr. Rahamin Haim",male,NaN,0,0,374746,8.0500,NaN,S
776,777,0,3,"Tobin, Mr. Roger",male,NaN,0,0,383121,7.7500,F38,Q
330,331,1,3,"McCoy, Miss. Agnes",female,NaN,2,0,367226,23.2500,NaN,Q


In [47]:
df.notnull().sum()

PassengerId    891
Survived       891
Pclass         891
Name           891
Sex            891
Age            714
SibSp          891
Parch          891
Ticket         891
Fare           891
Cabin          204
Embarked       889
dtype: int64

In [48]:
df.drop(columns=['PassengerId','Ticket','Name','Cabin'], inplace=True)

In [49]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [50]:
x_train,x_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']), df['Survived'], test_size=0.2,random_state=42)

In [51]:
x_train.head(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5,S
733,2,male,23.0,0,0,13.0,S


In [52]:
y_train.head(2)

331    0
733    0
Name: Survived, dtype: int64

In [53]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [54]:
si_age = SimpleImputer()
si_embarked = SimpleImputer(strategy='most_frequent')

x_train_age = si_age.fit_transform(x_train[['Age']])
x_train_embarked = si_embarked.fit_transform(x_train[['Embarked']])

x_test_age = si_age.transform(x_test[['Age']])
x_test_embarked = si_embarked.transform(x_test[['Embarked']])


In [55]:
print(pd.DataFrame(x_train_age).notnull().sum(),pd.DataFrame(x_train_embarked).notnull().sum(), sep = '\n')

0    712
dtype: int64
0    712
dtype: int64


In [56]:
ohe_sex = OneHotEncoder(sparse_output = False,handle_unknown='ignore')
ohe_embarked = OneHotEncoder(sparse_output = False,handle_unknown='ignore')

x_train_sex = ohe_sex.fit_transform(x_train[['Sex']])
x_train_embarked = ohe_embarked.fit_transform(x_train_embarked)

x_test_sex = ohe_sex.transform(x_test[['Sex']])
x_test_embarked = ohe_embarked.transform(x_test_embarked)


In [57]:
x_train_rem = x_train.drop(columns = ['Sex','Age','Embarked'])
x_test_rem = x_test.drop(columns = ['Sex','Age','Embarked'])

x_train_transformed = np.concatenate((x_train_rem,x_train_age,x_train_sex,x_train_embarked),axis = 1)
x_test_transformed = np.concatenate((x_test_rem,x_test_age,x_test_sex,x_test_embarked),axis = 1)



In [58]:
x_train_transformed.shape

(712, 10)

In [59]:
clf = DecisionTreeClassifier()
clf.fit(x_train_transformed,y_train)

DecisionTreeClassifier()

In [60]:
y_pred = clf.predict(x_test_transformed)

In [61]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.7932960893854749

In [63]:
import pickle
pickle.dump(ohe_sex,open('./models/ohe_sex.pkl','wb'))
pickle.dump(ohe_embarked,open('./models/ohe_embarked.pkl','wb'))
pickle.dump(clf,open('./models/clf.pkl','wb'))


